In [ ]:
%load_ext autoreload
%load_ext autotime

In [ ]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

In [ ]:
import dask.dataframe as dd
import numpy as np
import pandas as pd


In [ ]:
%autoreload 2

from wisio.recorder import RecorderAnalyzer

log_dir = "/p/gpfs1/iopp/recorder_app_logs/mummi-wemul/nodes-32-ppn-32/_parquet"

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
#         force_local=True,
        cluster_settings=dict(
#             cores=4,
            dashboard_port=3696,
            local_directory="/var/tmp/dask-recorder/cosmoflow",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
) 

recorder_analyzer

In [ ]:
[client.dashboard_link for client in recorder_analyzer.cluster_manager.clients]

In [ ]:
%autoreload 2

res = recorder_analyzer.analyze_parquet(log_dir=log_dir)

In [ ]:
views, bottlenecks = res

In [ ]:
views[('trange',)]['bottlenecks'].head()

In [ ]:
'file_id' in views[('trange',)]['expanded_view'].columns

In [ ]:
views[('file_id',)]['bottleneck_view'].head()

In [ ]:
from wisio._recorder.analysis import compute_unique_file_names, compute_unique_proc_names
%autoreload 2

unique_file_names = compute_unique_file_names(log_dir=log_dir)
unique_proc_names = compute_unique_proc_names(log_dir=log_dir)


In [ ]:
from wisio._recorder.bottlenecks import RecorderBottleneckDetector

bottleneck_detector = RecorderBottleneckDetector(
    log_dir=log_dir,
    views=views,
    view_types=['trange', 'file_id', 'proc_id'],
    unique_file_names=unique_file_names,
    unique_proc_names=unique_proc_names
)

bottlenecks = bottleneck_detector.detect_bottlenecks(max_io_time=max_io_time)

len(bottlenecks)


In [ ]:
bottlenecks[('proc_id', 'trange')]

In [ ]:
bottlenecks[('proc_id', 'trange', 'file_id')]

In [ ]:
views[('trange',)]['bottleneck_view'] \
    .groupby(['file_id', 'proc_id', 'trange']) \
    .first() \
    .drop(columns=['acc_pat', 'io_cat'], errors='ignore') \
    .merge(unique_file_names, left_index=True, right_index=True) \
    .merge(unique_proc_names, left_index=True, right_index=True) \
    .compute()